# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [4]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [31]:
X = adult_dt.drop(columns = 'income')
Y = adult_dt['income']

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

Random state is a parameter in the splitting function that controls the randomness in data splitting. It is useful because by setting this parameter to a specific number, one can reproduce the results by splitting, which helps evaluation of the model.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [34]:
#inspect columns and data types
print(adult_dt.dtypes)

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income             int64
dtype: object


In [41]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

num_cols = ['age',
       'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 
       ]
cat_cols = ['workclass',
       'education', 'marital-status', 'occupation', 'relationship',
       'race', 'sex','native-country',
       ]

pipe_num = Pipeline(
    [
        ('knnimputer', KNNImputer(n_neighbors=7, weights='distance')),
        ('robustscaler', RobustScaler()),
    ]
)

pipe_cat = Pipeline(
    [
        ('simpleimputer', SimpleImputer(strategy = 'most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore',drop='if_binary')),

    ]
)

cl_transformer= ColumnTransformer([
    ('num_transform', pipe_num, num_cols), 
    ('cat_transform', pipe_cat, cat_cols)
])

cl_transformer



ColumnTransformer(transformers=[('num_transform',
                                 Pipeline(steps=[('knnimputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transform',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [42]:
from sklearn.ensemble import RandomForestClassifier

pipe1 = Pipeline([
    ('preprocessing', cl_transformer),
    ('rfclf', RandomForestClassifier())
])

pipe1

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transform',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transform',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('rfclf', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [43]:
from sklearn.model_selection import cross_validate

scoring = ['neg_log_loss','roc_auc', 'accuracy', 'balanced_accuracy']

pipe1_dict = cross_validate(pipe1, X, Y, cv=5, scoring = scoring, return_train_score = True)

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [51]:
sorted_pipe1_cvscore = pd.DataFrame(pipe1_dict).sort_values(by='test_neg_log_loss')
sorted_pipe1_cvscore

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
1,6.204078,0.184287,-0.385725,-0.080328,0.896825,1.000000,0.848741,0.999923,0.764729,0.999841
0,6.105780,0.189945,-0.376711,-0.080240,0.900104,1.000000,0.852756,1.000000,0.770313,1.000000
4,6.031810,0.181430,-0.376141,-0.081189,0.906507,1.000000,0.858262,0.999962,0.782322,0.999920
3,6.121696,0.184009,-0.366012,-0.080562,0.903348,1.000000,0.859490,0.999962,0.778341,0.999920
2,6.160759,0.181951,-0.362411,-0.081383,0.903595,0.999999,0.851044,0.999923,0.775827,0.999895


Calculate the mean of each metric. 

In [53]:
sorted_pipe1_cvscore.mean()

fit_time                   6.124825
score_time                 0.184324
test_neg_log_loss         -0.373400
train_neg_log_loss        -0.080740
test_roc_auc               0.902076
train_roc_auc              1.000000
test_accuracy              0.854059
train_accuracy             0.999954
test_balanced_accuracy     0.774307
train_balanced_accuracy    0.999915
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [60]:
pipe1.fit(X_train, Y_train)

Y_pred_test = pipe1.predict(X_test)
Y_pred_proba_test = pipe1.predict_proba(X_test)

from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

performance_metrics = {
        'neg_log_loss_test': - log_loss(Y_test, Y_pred_proba_test),
        'roc_auc_test': roc_auc_score(Y_test, Y_pred_proba_test[:, 1]),
        'accuracy_score_test': accuracy_score(Y_test, Y_pred_test),
        'balanced_accuracy_score_test': balanced_accuracy_score(Y_test, Y_pred_test),
    }

performance_metrics


{'neg_log_loss_test': -0.3877760565653852,
 'roc_auc_test': 0.9003228242981368,
 'accuracy_score_test': 0.8525949431876344,
 'balanced_accuracy_score_test': 0.7727414037668825}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Answer:
By converting income into a binary numerical variable at the data loading stage, it ensures that the target variable is in a suitable form for both training and testing datasets. 
This transformation makes it convenient for modeling, asthe variable is consistently ready from the beginning.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.